In [0]:
# import libraries 
import pandas as pd
import numpy as np
from google.colab import files

In [38]:
# load FX price history data
# show first 5 rows
fx = pd.read_csv('https://raw.githubusercontent.com/AndrewMarksArt/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/master/historical_fx_rates.csv')
fx.head()

,DATE,DEXCHUS,DEXDNUS,DEXHKUS,DEXJPUS,DEXKOUS,DEXMXUS,DEXNOUS,DEXSDUS,DEXSIUS,DEXSZUS,DEXTAUS,DEXUSAL,DEXUSEU,DEXUSNZ,DEXUSUK
0,1971-01-04,NaN,7.4850,NaN,357.73,NaN,NaN,7.1359,5.1643,NaN,4.3180,NaN,1.1127,NaN,1.1138,2.3938
1,1971-01-05,NaN,7.4872,NaN,357.81,NaN,NaN,7.1422,5.1628,NaN,4.3117,NaN,1.1132,NaN,1.1143,2.3949
2,1971-01-06,NaN,7.4839,NaN,357.86,NaN,NaN,7.1367,5.1614,NaN,4.3113,NaN,1.1140,NaN,1.1151,2.3967
3,1971-01-07,NaN,7.4850,NaN,357.87,NaN,NaN,7.1386,5.1649,NaN,4.3103,NaN,1.1138,NaN,1.1150,2.3963
4,1971-01-08,NaN,7.4850,NaN,357.82,NaN,NaN,7.1382,5.1631,NaN,4.3109,NaN,1.1124,NaN,1.1154,2.3972


In [39]:
# load auction price history data
# drop un-needed columns and show first five rows
auction = pd.read_csv('https://raw.githubusercontent.com/AndrewMarksArt/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/master/Damien_Hirst_Auction_Results.csv')
auction.drop(['ID_Batch_Loop#', 'Artist_Name', 'Date_Created', 'Size'], axis=1, inplace=True)
auction = auction.rename(columns={'Sale_Date': 'Date'})
auction.head()

,Title,Medium,Signed_Edition,Estimate_Range,Realized_Price,Sale,Auction_House,Date
0,"SPOT DRAWING NO. 2, 1990",coloured pencil on paper,Signed,"80,000 - 120,000 GBP","175,000 GBP",Contemporary Art Day Auction,Sotheby's London,"Jun. 27, 2018"
1,"Supreme Skate Deck Red #7, 1987",Color silkscreen on plywood in original,from an edition of 500,"800 - 1,200 USD",450 USD*,"Public Art Auction: Modern & Contemporary Art,...",Santa Monica Auctions,"Jun. 04, 2017"
2,"Supreme Skate Deck Yellow #6, 1987",Color silkscreen on plywood in original bag,from an edition of 500,"800 - 1,200 USD",475 USD*,"Public Art Auction: Modern & Contemporary Art,...",Santa Monica Auctions,"Jun. 04, 2017"
3,"Pharmacy, circa1990",offset lithograph printed in colours,42/200,"1,500 - 2,000 GBP","1,500 GBP",Mixed Media: 20th Century Art,"Dreweatts & Bloomsbury Auctions, Bloomsbury House","Jul. 16, 2015"
4,"The Complete Spot Paintings, 1986-2011",with butterfly and heart drawing on the slipca...,Signed,"1,000 - 1,500 GBP","2,375 GBP",Prints & Multiples,Bonhams Knightsbridge,"Jun. 03, 2015"


In [0]:
# create var for the estimate range column from auction data
estimate = auction['Estimate_Range']

# create empty lists that will be used to create new columns 
low = []
high = []
currency = []

# loop through elements in the estimate range
# pull out low estimate, high estimate and currency
for i in range(len(estimate)):
  est = estimate[i]
  try:
    temp = est.split()
    if len(temp) == 4:
      low.append(temp[0])
      high.append(temp[2])
      currency.append(temp[3])
    elif len(temp) == 2:
      low.append(temp[0])
      high.append(np.nan)
      currency.append(temp[1])
    else:
      low.append(np.nan)
      high.append(np.nan)
      currency.append(np.nan)
  except:
    pass

# use the lists to create new columns for the auction data 
auction['Low Estimate'] = low
auction['High Estimate'] = high
auction['Currency'] = currency

# drop estimate range since it is now redundant 
auction.drop(['Estimate_Range'], axis=1, inplace=True)

In [0]:
# create var to hold realized prices from aution data
prices = auction['Realized_Price']

# loop through realized prices to remove currency codes
for i in range(len(prices)):
  try:
    temp = prices[i].split()
    if len(temp) < 2:
      prices[i] = -1
    else:
      prices[i] = temp[0]
  except:
    pass
  
# replace realized prices so new values have no currency codes
auction['Realized_Price'] = prices

In [0]:
# set var for currency column
currency = auction['Currency']

# loop through currency and remove * and Sold values
for i in range(len(currency)):
  try:
    if '*' in currency[i]:
      currency[i] = 'EUR'
    if 'Sold' in currency[i]:
      currency[i] = np.nan
  except:
    pass

# replace currency with new list that removed * and Sold
auction['Currency'] = currency

In [0]:
# fill remaining NaN's in currency column with 'USD'
auction['Currency'].fillna('USD', inplace=True)

In [0]:
# set up dictionary with currency code as keys and country names as the values
country_dict = {'GBP': 'United Kingdom', 'USD': 'United States', 'EUR': 'Europe', 'HKD': 'Hong Kong', 'CNY': 'China', 'CHF': 'Switzerland',
             'SEK': 'Sweden', 'JPY': 'Japan', 'KRW': 'South Korea', 'AUD': 'Australia', 'NOK': 'Norway', 'ZAR': 'South Africa', 'DKK': 'Denmark', 'NZD': 'New Zealand', 
               'TWD': 'Taiwan', 'SGD': 'Singapore', 'MXN': 'Mexico', 'CZK': 'Czech Republic'}

In [0]:
# set currency column to new var and create list for countries
currency = auction['Currency']
country = []

# loop through currency column and get the country name that corresponds to the currency code 
for i in range(len(currency)):
  try:
    country.append(country_dict.get(currency[i]))
  except:
    pass
    
# create a county column with the counrty list
auction['Country'] = country

In [0]:
# replace no sale date listed with NaN
sale = auction['Date']
for i in range(len(sale)):
  try:
    if 'no' in sale[i]:
      sale[i] = np.nan
  except:
    pass

In [0]:
# make sure date column is a DateTime type
auction['Date'] = pd.to_datetime(auction['Date'])

In [48]:
# create a new df with the date and country from the auction data
auction_location = auction[['Date', 'Country']]
auction_location.head()

,Date,Country
0,2018-06-27,United Kingdom
1,2017-06-04,United States
2,2017-06-04,United States
3,2015-07-16,United Kingdom
4,2015-06-03,United Kingdom


## Want to refactor and write a function to handle this when time allows, don't like repeating so much

*   take the fx df and break out each country, add the country name, and chage the column names to 'Date', 'Value', and 'Country' into there own df.
*  Make sure all FX rates are in currency / USD ratio.
*  Concat all df's into one df



In [49]:
china = fx[['DATE', 'DEXCHUS']]
china['Country'] = 'China'
china = china.rename(columns={'DATE': 'Date', 'DEXCHUS': 'Value'})
china = china[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
denmark = fx[['DATE', 'DEXDNUS']]
denmark['Country'] = 'Denmark'
denmark = denmark.rename(columns={'DATE': 'Date', 'DEXDNUS': 'Value'})
denmark = denmark[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [51]:
hong_kong = fx[['DATE', 'DEXHKUS']]
hong_kong['Country'] = 'Hong Kong'
hong_kong = hong_kong.rename(columns={'DATE': 'Date', 'DEXHKUS': 'Value'})
hong_kong = hong_kong[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [52]:
japan = fx[['DATE', 'DEXJPUS']]
japan['Country'] = 'Japan'
japan = japan.rename(columns={'DATE': 'Date', 'DEXJPUS': 'Value'})
japan = japan[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [53]:
south_korea = fx[['DATE', 'DEXKOUS']]
south_korea['Country'] = 'South Korea'
south_korea = south_korea.rename(columns={'DATE': 'Date', 'DEXKOUS': 'Value'})
south_korea = south_korea[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [54]:
mexico = fx[['DATE', 'DEXMXUS']]
mexico['Country'] = 'Mexico'
mexico = mexico.rename(columns={'DATE': 'Date', 'DEXMXUS': 'Value'})
mexico = mexico[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [55]:
norway = fx[['DATE', 'DEXNOUS']]
norway['Country'] = 'Norway'
norway = norway.rename(columns={'DATE': 'Date', 'DEXNOUS': 'Value'})
norway = norway[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [56]:
sweden = fx[['DATE', 'DEXSDUS']]
sweden['Country'] = 'Sweden'
sweden = sweden.rename(columns={'DATE': 'Date', 'DEXSDUS': 'Value'})
sweden = sweden[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [57]:
singapore = fx[['DATE', 'DEXSIUS']]
singapore['Country'] = 'Singapore'
singapore = singapore.rename(columns={'DATE': 'Date', 'DEXSIUS': 'Value'})
singapore = singapore[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [58]:
switzerland = fx[['DATE', 'DEXSZUS']]
switzerland['Country'] = 'Switzerland'
switzerland = switzerland.rename(columns={'DATE': 'Date', 'DEXSZUS': 'Value'})
switzerland = switzerland[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [59]:
taiwan = fx[['DATE', 'DEXTAUS']]
taiwan['Country'] = 'Taiwan'
taiwan = taiwan.rename(columns={'DATE': 'Date', 'DEXTAUS': 'Value'})
taiwan = taiwan[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
australia = fx[['DATE', 'DEXUSAL']]
australia['Country'] = 'Australia'
australia = australia.rename(columns={'DATE': 'Date', 'DEXUSAL': 'Value'})
australia['Value'] = australia['Value'].apply(lambda x: 1/x)
australia = australia[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [61]:
europe = fx[['DATE', 'DEXUSEU']]
europe['Country'] = 'Europe'
europe = europe.rename(columns={'DATE': 'Date', 'DEXUSEU': 'Value'})
europe['Value'] = europe['Value'].apply(lambda x: 1/x)
europe = europe[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [62]:
new_zealand = fx[['DATE', 'DEXUSNZ']]
new_zealand['Country'] = 'New Zealand'
new_zealand = new_zealand.rename(columns={'DATE': 'Date', 'DEXUSNZ': 'Value'})
new_zealand['Value'] = new_zealand['Value'].apply(lambda x: 1/x)
new_zealand = new_zealand[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [63]:
united_kingdom = fx[['DATE', 'DEXUSUK']]
united_kingdom['Country'] = 'United Kingdom'
united_kingdom = united_kingdom.rename(columns={'DATE': 'Date', 'DEXUSUK': 'Value'})
united_kingdom['Value'] = united_kingdom['Value'].apply(lambda x: 1/x)
united_kingdom = united_kingdom[['Date', 'Country', 'Value']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [64]:
results = pd.concat([china, denmark, hong_kong, japan, south_korea, mexico,
                     norway, sweden, singapore, switzerland, taiwan, australia,
                     europe, new_zealand, united_kingdom])
results.shape

(189675, 3)

In [0]:
results['Date'] = pd.to_datetime(results['Date'])

In [66]:
results.dtypes

Date       datetime64[ns]
Country            object
Value             float64
dtype: object

In [67]:
merged = pd.merge(auction_location,
                  results,
                  on=['Date', 'Country'],
                  how='left')
merged.head()

,Date,Country,Value
0,2018-06-27,United Kingdom,0.761093
1,2017-06-04,United States,NaN
2,2017-06-04,United States,NaN
3,2015-07-16,United Kingdom,0.640985
4,2015-06-03,United Kingdom,0.651423


In [68]:
merged.shape

(6541, 3)

In [69]:
countries = merged['Country'].unique()
countries

array(['United Kingdom', 'United States', 'Europe', 'Hong Kong', 'China',
       'Switzerland', 'Sweden', 'Japan', 'South Korea', 'Australia',
       'Norway', 'South Africa', 'Denmark', 'New Zealand', 'Taiwan',
       'Singapore', 'Mexico', 'Czech Republic'], dtype=object)

In [72]:
merged.isnull().sum()

Date         51
Country       0
Value      2659
dtype: int64

In [0]:
# clean up FX data to match dates needed for the auction data
df = pd.DataFrame()
count = 0

# Set value to 1 if US
# Drop missing dates 
# Forward fill NaN values for the 'Value' column
for i in range(len(countries)):
  country = countries[i]
  temp = merged[merged['Country'] == country]
  
  if country == 'United States':
    temp['Value'] = 1
  
  temp = temp.dropna(subset=['Date'])
  temp = temp.fillna(method='ffill')
  
  if count == 0:
    df = temp
    count += 1
  else:
    df = pd.concat([df, temp])
  

In [74]:
df.head()

,Date,Country,Value
0,2018-06-27,United Kingdom,0.761093
3,2015-07-16,United Kingdom,0.640985
4,2015-06-03,United Kingdom,0.651423
5,2015-03-10,United Kingdom,0.662866
6,2014-10-23,United Kingdom,0.623558


In [75]:
df.shape

(6490, 3)

In [76]:
df.isnull().sum()

Date        0
Country     0
Value      32
dtype: int64

In [0]:
# save data frame as csv and download the file
file_name = 'cleaned_fx_data.csv'
export_csv = df.to_csv(file_name, index=None, header=True)
files.download('cleaned_fx_data.csv')